| num_modelo | caracteristicas | epochs train | mejor epoch | rmse validation | rmse test |
|------------|----------------|--------------|-------------|----------------|-----------|
| 1 | AA | 706 | 456  | 0.8238750774549344 | 0.9283088700606827 |
| 2 | AA | 597 | 347  | 0.7000741553947083 | 0.7731256329514212 |
| 3 | AA | 643 | 393  | 0.6305856704941186 | 0.7298970985210189 |
| 4 | AA | AA | AA | AA | AA |
| 5 | AA | AA | AA | AA | AA |


# Modelos

## Modelo 1

In [ ]:
from torch.nn import Module
from torch_geometric.nn import GCN, MLP, global_add_pool, GCNConv, global_max_pool, global_mean_pool, BatchNorm

class GCNGraph(Module):

    def __init__(self, num_features, num_classes, hidden_channels=128, n_layers=4, dropout=0.5):
        super(GCNGraph, self).__init__()

        # Proyección inicial (proyectar las 9 features a un espacio latente común)
        self.input_proj = torch.nn.Linear(num_features, hidden_channels)
        self.input_norm = torch.nn.BatchNorm1d(hidden_channels)

        # GCN explícito por capas
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        self.dropout = dropout
        for _ in range(n_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.norms.append(BatchNorm(hidden_channels))

        # Pooling múltiple
        self.pool_dim = hidden_channels * 2

        # MLP profundo
        self.pool_mlp = torch.nn.Sequential(
            torch.nn.Linear(self.pool_dim, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(hidden_channels),
            torch.nn.Dropout(dropout)
        )

        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, hidden_channels//2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(hidden_channels//2),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels//2, num_classes)
        )

    def reset_parameters(self):
        self.input_proj.reset_parameters()
        self.input_norm.reset_parameters()

        for conv, norm in zip(self.convs, self.norms):
            conv.reset_parameters()
            norm.reset_parameters()
        for m in self.pool_mlp:
            if hasattr(m, "reset_parameters"):
                m.reset_parameters()
        for m in self.regressor:
            if hasattr(m, "reset_parameters"):
                m.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Proyección inicial
        x = self.input_proj(x)        
        x = self.input_norm(x)

        # GCN profundo con residual
        for conv, norm in zip(self.convs, self.norms):
            h = conv(x, edge_index)
            h = norm(h)
            h = torch.nn.functional.leaky_relu(h, negative_slope=0.1)
            h = torch.nn.functional.dropout(h, p=self.dropout, training=self.training)
            x = torch.nn.functional.layer_norm(x + h, x.size()[1:])  # residual + layernorm

        # Pooling
        x = torch.cat([
            global_add_pool(x, batch),
            global_max_pool(x, batch),
        ], dim=1)

        # Proyección
        x = self.pool_mlp(x)

        # Regresión
        out = self.regressor(x)
        out = torch.clamp(out, min=-3.0, max=7.0)
        return out


## Modelo 2

In [ ]:
from torch.nn import Module
from torch_geometric.nn import GCN, MLP, global_add_pool, GCNConv, global_max_pool, global_mean_pool, BatchNorm

class GCNGraph(Module):

    def __init__(self, num_features, num_classes, hidden_channels=128, n_layers=4, dropout=0.4):
        super(GCNGraph, self).__init__()

        # Proyección inicial (proyectar las 9 features a un espacio latente común)
        self.input_proj = torch.nn.Linear(num_features, hidden_channels)
        self.input_norm = torch.nn.BatchNorm1d(hidden_channels)

        # GCN explícito por capas
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        self.n_layers = n_layers
        self.dropout = dropout
        for _ in range(n_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.norms.append(BatchNorm(hidden_channels))

        # Pooling múltiple
        self.pool_dim = hidden_channels * 2

        # MLP profundo
        self.pool_mlp = torch.nn.Sequential(
            torch.nn.Linear(self.pool_dim, hidden_channels),
            torch.nn.LeakyReLU(negative_slope=0.1),
            torch.nn.LayerNorm(hidden_channels),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels, hidden_channels // 2),
            torch.nn.LeakyReLU(negative_slope=0.1),
            torch.nn.LayerNorm(hidden_channels // 2),
            torch.nn.Dropout(dropout)
        )

        self.regressor = torch.nn.Linear(hidden_channels // 2, num_classes)


    def reset_parameters(self):
        self.input_proj.reset_parameters()
        self.input_norm.reset_parameters()

        for conv, norm in zip(self.convs, self.norms):
            conv.reset_parameters()
            norm.reset_parameters()

        for m in self.pool_mlp:
            if hasattr(m, "reset_parameters"):
                m.reset_parameters()
        for m in self.regressor:
            if hasattr(m, "reset_parameters"):
                self.regressor.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Proyección inicial
        x = self.input_proj(x)        
        x = self.input_norm(x)

        # GCN profundo con residual
        for conv, norm in zip(self.convs, self.norms):
            h = conv(x, edge_index)
            h = norm(h)
            h = torch.nn.functional.leaky_relu(h, negative_slope=0.1)
            h = torch.nn.functional.dropout(h, p=self.dropout, training=self.training)
            x = torch.nn.functional.layer_norm(x + h, x.size()[1:])

        # Pooling
        x = torch.cat([
            global_mean_pool(x, batch),
            global_max_pool(x, batch),
        ], dim=1)

        # Proyección
        x = self.pool_mlp(x)

        # Regresión
        out = self.regressor(x)
        out = torch.clamp(out, min=-3.0, max=7.0)
        return out


## Modelo 3

In [ ]:
from torch.nn import Module
from torch_geometric.nn import GCN, MLP, global_add_pool, GCNConv, global_max_pool, global_mean_pool, BatchNorm

class GCNGraph(Module):

    def __init__(self, num_features, num_classes, hidden_channels=256, n_layers=4, dropout=0.25):
        super(GCNGraph, self).__init__()

        self.dropout = dropout

        # Proyección inicial (proyectar las 9 features a un espacio latente común)
        self.input_proj = torch.nn.Linear(num_features, hidden_channels)
        self.input_norm = torch.nn.BatchNorm1d(hidden_channels)

        # GCN explícito por capas
        self.convs = torch.nn.ModuleList()
        self.norms = torch.nn.ModuleList()
        for _ in range(n_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.norms.append(BatchNorm(hidden_channels))

        # Pooling múltiple
        self.pool_dim = hidden_channels * 2

        # MLP profundo
        self.pool_mlp = torch.nn.Sequential(
            torch.nn.Linear(self.pool_dim, hidden_channels),
            torch.nn.LeakyReLU(negative_slope=0.1),
            torch.nn.LayerNorm(hidden_channels),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels, hidden_channels),
            torch.nn.LeakyReLU(negative_slope=0.1),
            torch.nn.LayerNorm(hidden_channels),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_channels, hidden_channels // 2),
            torch.nn.LeakyReLU(negative_slope=0.1),
            torch.nn.LayerNorm(hidden_channels // 2),
            torch.nn.Dropout(dropout)
        )

        self.regressor = torch.nn.Linear(hidden_channels // 2, num_classes)


    def reset_parameters(self):
        self.input_proj.reset_parameters()
        self.input_norm.reset_parameters()

        for conv, norm in zip(self.convs, self.norms):
            conv.reset_parameters()
            norm.reset_parameters()

        for m in self.pool_mlp:
            if hasattr(m, "reset_parameters"):
                m.reset_parameters()
        for m in self.regressor:
            if hasattr(m, "reset_parameters"):
                self.regressor.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Proyección inicial
        x = self.input_proj(x)        
        x = self.input_norm(x)

        # GCN profundo con residual
        for conv, norm in zip(self.convs, self.norms):
            h = conv(x, edge_index)
            h = norm(h)
            h = torch.nn.functional.leaky_relu(h, negative_slope=0.1)
            h = torch.nn.functional.dropout(h, p=self.dropout, training=self.training)
            x = torch.nn.functional.layer_norm(x + h, x.size()[1:])

        # Pooling
        x = torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], dim=1)

        # Proyección
        x = self.pool_mlp(x)

        # Regresión
        out = self.regressor(x)
        out = torch.clamp(out, min=-3.0, max=7.0)
        return out


## Modelo 4

## Modelo 5